/* Here is the Country Club, SQL, mini project. This project is partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.
PART 1: PHPMyAdmin
Logged into PHPMyAdmin at https://sql.springboard.com/phpmyadmin/
with a provided Username and Password:
    
The data is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

/* QUESTIONS */
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name
FROM Facilities
where membercost > 0     


/* Q2: How many facilities do not charge a fee to members? */

SELECT count(name)
FROM Facilities
where membercost <= 0

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
where membercost > 0
AND membercost < monthlymaintenance * .20

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM Facilities
WHERE facid IN (1,4)

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT 
	name, 
	monthlymaintenance,
	CASE WHEN monthlymaintenance > 100 
		THEN 'expensive'
		ELSE 'cheap' END
		AS cost_label
FROM Facilities
ORDER BY monthlymaintenance DESC;

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT 
	firstname, 
	surname,
	cast(joindate AS date) joindate
FROM Members
WHERE joindate IN (SELECT MAX(joindate) from Members)

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT 
  DISTINCT subquery.mem_name,
  subquery.facility_name
FROM (
  SELECT
    Facilities.name as facility_name,
    CONCAT(Members.firstname,' ',Members.surname) AS mem_name
  FROM Bookings
  LEFT JOIN Facilities
  ON Bookings.facid = Facilities.facid
  LEFT JOIN Members
  ON Bookings.memid = Members.memid
  WHERE Bookings.facid < 2
) as subquery
ORDER BY subquery.mem_name

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT 
  	Facilities.name as facility_name,
	CONCAT(Members.firstname,' ',Members.surname) AS member_name,
	CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost*Bookings.slots
         ELSE Facilities.membercost*Bookings.slots END as cost
FROM Bookings
LEFT JOIN Facilities
	ON Bookings.facid = Facilities.facid
LEFT JOIN Members
  	ON Bookings.memid = Members.memid
WHERE DATE_FORMAT(Bookings.starttime, '%Y-%m-%d') = '2012-09-14'
HAVING cost > 30
ORDER BY cost DESC

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT 
	subquery.facility_name,  
	subquery.mem_name,
	subquery.cost
FROM (
  SELECT
    Facilities.name AS facility_name,
    CONCAT(Members.firstname,' ',Members.surname) AS mem_name,
    CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost*Bookings.slots
         ELSE Facilities.membercost*Bookings.slots END AS cost
  FROM Bookings
  LEFT JOIN Facilities
  	ON Bookings.facid = Facilities.facid
  LEFT JOIN Members
  	ON Bookings.memid = Members.memid
  WHERE DATE_FORMAT(Bookings.starttime, '%Y-%m-%d') = '2012-09-14'
  HAVING cost > 30
) AS subquery
ORDER BY cost DESC


In [1]:
# For the remaining questions, the queries are run within the notebook
# connected to local sqlite3 database on machine
import pandas as pd
import numpy as np
import sqlite3 as sql

# create db connection
database = '/Users/robertspoonmore/Desktop/Springboard information/Country Club Case Study/SQLFiles Tier 2/sqlite_db_pythonsqlite.db'
connection = sql.connect(database)

In [2]:
# Show head information for Table Bookings 
query = '''SELECT * 
            FROM Bookings'''

df = pd.read_sql_query(query, connection)
df.head(50)

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1
5,5,8,1,2012-07-03 15:00:00,1
6,6,0,2,2012-07-04 09:00:00,3
7,7,0,2,2012-07-04 15:00:00,3
8,8,4,3,2012-07-04 13:30:00,2
9,9,4,0,2012-07-04 15:00:00,2


In [3]:
# Show head information for Table Bookings 
query = '''SELECT * 
            FROM Facilities'''

df = pd.read_sql_query(query, connection)
df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [4]:
# Show head information for Table Bookings 
query = '''SELECT * 
            FROM Members'''

df = pd.read_sql_query(query, connection)
df.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [5]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members! 
query10 = '''SELECT 
            Facilities.name as facility_name,
            CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost*Bookings.slots
                 ELSE Facilities.membercost*Bookings.slots END as revenue
            FROM Bookings
            LEFT JOIN Facilities
                ON Bookings.facid = Facilities.facid
            LEFT JOIN Members
                ON Bookings.memid = Members.memid
            GROUP BY facility_name
            HAVING revenue < 1000 AND revenue > 0
            ORDER BY revenue DESC'''

df = pd.read_sql_query(query10, connection)
df.head(10)

,facility_name,revenue
0,Massage Room 2,160.0
1,Tennis Court 2,75.0
2,Squash Court,35.0
3,Massage Room 1,19.8
4,Tennis Court 1,15.0


In [6]:
# Q11: Produce a report of members and who recommended them 
# in alphabetic surname,firstname order 
query11 = '''SELECT DISTINCT
                m1.surname || ',' || m1.firstname AS member_name,
                m2.surname || ',' || m2.firstname AS recommending_name
            FROM Members as m1
            INNER JOIN Members as m2
            ON m1.recommendedby = m2.memid
            ORDER BY member_name'''

df = pd.read_sql_query(query11, connection)
df.head(20)


,member_name,recommending_name
0,"Bader,Florence","Stibbons,Ponder"
1,"Baker,Anne","Stibbons,Ponder"
2,"Baker,Timothy","Farrell,Jemima"
3,"Boothe,Tim","Rownam,Tim"
4,"Butters,Gerald","Smith,Darren"
5,"Coplin,Joan","Baker,Timothy"
6,"Crumpet,Erica","Smith,Tracy"
7,"Dare,Nancy","Joplette,Janice"
8,"Genting,Matthew","Butters,Gerald"
9,"Hunt,John","Purview,Millicent"


In [7]:
# Q12: Find the facilities with their usage by member, but not guests
query12 = '''SELECT 
                name,
                COUNT(name) AS member_use_count
            FROM Bookings AS b
            LEFT JOIN Facilities AS f
                USING(facid)
            WHERE b.memid > 0
            GROUP BY name
            ORDER BY member_use_count DESC'''

df = pd.read_sql_query(query12, connection)
df.head(10)

,name,member_use_count
0,Pool Table,783
1,Snooker Table,421
2,Massage Room 1,421
3,Table Tennis,385
4,Badminton Court,344
5,Tennis Court 1,308
6,Tennis Court 2,276
7,Squash Court,195
8,Massage Room 2,27


In [8]:
# Q13: Find the facilities usage by month, but not guests
query13 = '''SELECT 
                name,
                strftime('%m', starttime) AS use_month,
                COUNT(name) AS member_use_count
            FROM Bookings AS b
            LEFT JOIN Facilities AS f
                USING(facid)
            WHERE b.memid > 0
            GROUP BY use_month'''

df = pd.read_sql_query(query13, connection)
df.head(10)

,name,use_month,member_use_count
0,Table Tennis,07,480
1,Tennis Court 1,08,1168
2,Tennis Court 1,09,1512
